In [2]:
import pandas as pd
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.utils import parse_netmhcpan_header, filter_rank, get_filtered_df
df = pd.read_csv('../29165_NetMHCpan.xls', header=[0,1], sep = '\t')
df = df.drop(index = df.sample(3).index)
df.columns = parse_netmhcpan_header(df.columns)
df.set_index(('base', 'Peptide'), inplace=True)
df.index.name = 'Peptide'

pep = pd.read_csv('../output/test_pep/9mers_Human_split_0.pep', header = None)
pep.columns = ['Peptide']


In [3]:
display(df.head(5))
display(pep.head(5))

base          HLA-A01:01                                        \
           Pos       ID       core      icore EL-score  EL_Rank BA-score   
Peptide                                                                    
MSSHEGGKK    0  PEPLIST  MSSHEGGKK  MSSHEGGKK   0.0103   4.8079   0.1270   
SHEGGKKKA    0  PEPLIST  SHEGGKKKA  SHEGGKKKA   0.0002  41.1667   0.0133   
HEGGKKKAL    0  PEPLIST  HEGGKKKAL  HEGGKKKAL   0.0006  24.0652   0.0120   
EGGKKKALK    0  PEPLIST  EGGKKKALK  EGGKKKALK   0.0002  39.9231   0.0191   
KKKALKQPK    0  PEPLIST  KKKALKQPK  KKKALKQPK   0.0000  81.2500   0.0067   

                   HLA-A03:01             ...                   HLA-A26:01  \
           BA_Rank       core      icore  ... BA-score  BA_Rank       core   
Peptide                                   ...                                
MSSHEGGKK   4.2039  MSSHEGGKK  MSSHEGGKK  ...   0.3113   2.3594  MSSHEGGKK   
SHEGGKKKA  89.0327  SHEGGKKKA  SHEGGKKKA  ...   0.0122  91.3582  SHEGGKKKA   
HEGGKKKAL  91.3973  HEGGKKKAL  HEGGKKKAL  ...   0.0091  95.4398  HEGGKKKAL   
EGGKKKALK  75.5713  EGGKKKALK  EGGKKKALK  ...   0.0564  28.0659  EGGKKKALK   
KKKALKQPK  96.6581  KKKALKQPK  KKKALKQPK  ...   0.0838  16.5062  KKKALKQPK   

                                                             end     
               icore EL-score  EL_Rank BA-score  BA_Rank     Ave NB  
Peptide                                                              
MSSHEGGKK  MSSHEGGKK   0.0009  15.5512   0.0628  12.3067  0.0289  0  
SHEGGKKKA  SHEGGKKKA   0.0000  57.1429   0.0068  92.4858  0.0001  0  
HEGGKKKAL  HEGGKKKAL   0.0010  14.6818   0.0165  60.3989  0.0006  0  
EGGKKKALK  EGGKKKALK   0.0001  42.4000   0.0128  72.1920  0.0006  0  
KKKALKQPK  KKKALKQPK   0.0000  54.0909   0.0070  91.8608  0.0019  0  

[5 rows x 22 columns]

,Peptide
0,MSSHEGGKK
1,SSHEGGKKK
2,SHEGGKKKA
3,HEGGKKKAL
4,EGGKKKALK


In [20]:
hlas = set([x for x in df.columns.get_level_values(0) if 'hla' in x.lower()])
ranks = [x for x in df.columns if x[0] in hlas and x[1] == 'EL_Rank' or x[1] == 'BA_Rank']
out = pd.merge(df[ranks].idxmin(axis=1).apply(lambda x: x[0]).rename('HLA'), df[ranks].min(axis=1).rename('xd'), left_index = True, right_index=True)

In [21]:
def return_columns(row, df):
    return [x for x in df.columns if x[0] == row['HLA']]

In [22]:
filtered = out.apply(lambda x: df.loc[x.name, return_columns(x, df)].values, axis=1)

In [ ]:
vals = pd.DataFrame.from_dict(dict(zip(filtered.index, filtered.values))).T
vals.index.name = 'Peptide'
vals.columns = ['core', 'icore', 'EL_score', 'EL_rank', 'BA_score', 'BA_rank']
vals

,core,icore,EL_score,EL_rank,BA_score,BA_rank
Peptide,,,,,,
MSSHEGGKK,MSSHEGGKK,MSSHEGGKK,0.0754,2.2719,0.3113,2.3594
SHEGGKKKA,SHEGGKKKA,SHEGGKKKA,0.0002,41.1667,0.0133,89.0327
HEGGKKKAL,HEGGKKKAL,HEGGKKKAL,0.001,14.6818,0.0165,60.3989
EGGKKKALK,EGGKKKALK,EGGKKKALK,0.0014,12.034,0.0564,28.0659
GGKKKALKQ,GGKKKALKQ,GGKKKALKQ,0.0002,24.48,0.0244,67.2066
GKKKALKQP,GKKKALKQP,GKKKALKQP,0.0,68.75,0.0063,96.819
KKKALKQPK,KKKALKQPK,KKKALKQPK,0.0055,7.1007,0.0838,16.5062
ALKQPKKQA,ALKQPKKQA,ALKQPKKQA,0.0034,8.6334,0.0479,34.5175


In [27]:
out.drop(columns=['xd']).merge(vals, left_index=True, right_index=True)

,HLA,core,icore,EL_score,EL_rank,BA_score,BA_rank
Peptide,,,,,,,
MSSHEGGKK,HLA-A03:01,MSSHEGGKK,MSSHEGGKK,0.0754,2.2719,0.3113,2.3594
SHEGGKKKA,HLA-A01:01,SHEGGKKKA,SHEGGKKKA,0.0002,41.1667,0.0133,89.0327
HEGGKKKAL,HLA-A26:01,HEGGKKKAL,HEGGKKKAL,0.001,14.6818,0.0165,60.3989
EGGKKKALK,HLA-A03:01,EGGKKKALK,EGGKKKALK,0.0014,12.034,0.0564,28.0659
GGKKKALKQ,HLA-A03:01,GGKKKALKQ,GGKKKALKQ,0.0002,24.48,0.0244,67.2066
GKKKALKQP,HLA-A03:01,GKKKALKQP,GKKKALKQP,0.0,68.75,0.0063,96.819
KKKALKQPK,HLA-A03:01,KKKALKQPK,KKKALKQPK,0.0055,7.1007,0.0838,16.5062
ALKQPKKQA,HLA-A03:01,ALKQPKKQA,ALKQPKKQA,0.0034,8.6334,0.0479,34.5175


In [37]:
import pandas as pd
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.utils import parse_netmhcpan_header
df = pd.read_csv('../29165_NetMHCpan.xls', header=[0,1], sep = '\t')
df = df.drop(index = df.sample(3).index)
df.columns = parse_netmhcpan_header(df.columns)
pep = pd.read_csv('../output/test_pep/9mers_Human_split_0.pep', header = None)
pep.columns = ['Peptide']
df.set_index(('base', 'Peptide'), inplace=True)
df.index.name = 'Peptide'

In [45]:
get_filtered_df(df_output, df).to_csv('../results.csv')

In [44]:
strx='../output/test_pep/9mers_Human_split_0.pep'
strx[strx.rfind('/')+1:strx.find('.pep')]

'9mers_Human_split_0'

In [48]:
df_output

,HLA,tmp
Peptide,,
MSSHEGGKK,HLA-A03:01,2.2719
SSHEGGKKK,HLA-A03:01,0.9013
SHEGGKKKA,HLA-A01:01,41.1667
HEGGKKKAL,HLA-A26:01,14.6818
GGKKKALKQ,HLA-A03:01,24.4800
KKALKQPKK,HLA-A03:01,3.7122
KALKQPKKQ,HLA-A03:01,12.7585
ALKQPKKQA,HLA-A03:01,8.6334


In [47]:
pd.read_csv(os.path.join(os.getcwd(), '../output/test_pep/9mers_Human_split_0.pep'))

,MSSHEGGKK
0,SSHEGGKKK
1,SHEGGKKKA
2,HEGGKKKAL
3,EGGKKKALK
4,GGKKKALKQ
5,GKKKALKQP
6,KKKALKQPK
7,KKALKQPKK
8,KALKQPKKQ
9,ALKQPKKQA


# script debug

In [ ]:
import pandas as pd
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.utils import parse_netmhcpan_header, filter_rank, get_filtered_df
df = pd.read_csv('../29165_NetMHCpan.xls', header=[0,1], sep = '\t')
df = df.drop(index = df.sample(3).index)
df.columns = parse_netmhcpan_header(df.columns)
df.set_index(('base', 'Peptide'), inplace=True)
df.index.name = 'Peptide'

pep = pd.read_csv('../output/test_pep/9mers_Human_split_0.pep', header = None)
pep.columns = ['Peptide']


In [4]:
os.listdir('../test_output/')

['8mers_netmhcpan_out.xls',
 '8mers_test_run_1.pep',
 '8mers_test_run_1.txt',
 'test_pep']

In [46]:
peps = pd.read_csv('../test_output/8mers_test_run_1.pep', header=None)
peps.columns = ['Peptide']

mhc = pd.read_csv('../test_output/8mers_netmhcpan_out.xls', header = [0,1], sep = '\t').drop_duplicates()
mhc.columns = parse_netmhcpan_header(mhc.columns)
mhc.set_index(('base', 'Peptide'), inplace=True)
mhc.index.name = 'Peptide'

In [53]:
peptxt = pd.read_csv('../test_output/8mers_test_run_1.txt')
peptxt.loc[peptxt.duplicated('peptide', keep = False)].sort_values('peptide')

,peptide,start_pos,uniprot_id,id_position
290,AAVREVYE,61,A0A024RBG1,A0A024RBG1_61
83,AAVREVYE,59,A0A024RBG1,A0A024RBG1_59
299,AGVKGKLG,70,A0A024RBG1,A0A024RBG1_70
92,AGVKGKLG,68,A0A024RBG1,A0A024RBG1_68
84,AVREVYEE,60,A0A024RBG1,A0A024RBG1_60
...,...,...,...,...
89,YEEAGVKG,65,A0A024RBG1,A0A024RBG1_65
119,YVLTVTEI,95,A0A024RBG1,A0A024RBG1_95
325,YVLTVTEI,96,A0A024RBG1,A0A024RBG1_96
117,YVYVLTVT,93,A0A024RBG1,A0A024RBG1_93


In [47]:
len(peps), len(mhc)

(402, 354)

In [ ]:
filtered = df_out.apply(lambda x: df_netmhcpan.loc[x.name, return_columns(x, df_netmhcpan)].values, axis=1)
# reshapes the filtered df
df_values = pd.DataFrame.from_dict(dict(zip(filtered.index, filtered.values))).T
df_values.index.name = 'Peptide'
df_values.columns = ['core', 'icore', 'EL_score', 'EL_rank', 'BA_score', 'BA_rank']
# Returns the output merged with the filtered values
return df_out.drop(columns = ['tmp']).merge(df_values, left_index=True, right_index=True)

In [15]:

def return_columns(row, df):
    """
    Returns the columns with HLA in it for multi indexing of netmhcpan xls df
    """
    return [x for x in df.columns if x[0] == row['HLA']]


In [44]:
df_out = filter_rank(mhc, 'el_rank').query('tmp<20.0')
df_out.apply(lambda x: mhc.loc[x.name, return_columns(x, mhc)].values, axis=1)

Peptide
AIXFXAKA    [AI-XFXAKA, AIXFXAKA, 0.0009, 18.5081, 0.0626,...
ALALFXAL    [ALA-LFXAL, ALALFXAL, 0.0345, 3.6047, 0.3462, ...
ALFAFWGX    [ALF-AFWGX, ALFAFWGX, 0.0013, 16.1, 0.1964, 10...
ALFXALFA    [ALFXALF-A, ALFXALFA, 0.0397, 3.3418, 0.4273, ...
APWGGAAV    [APW-GGAAV, APWGGAAV, 0.4415, 0.4625, 0.5616, ...
                                  ...                        
YPDQWIVP    [YPDQWIVP-, YPDQWIVP, 0.0048, 7.3065, 0.1243, ...
YVLTVTEI    [[YVL-TVTEI, YVLTVTEI, 0.0167, 5.2287, 0.2134,...
YVLTVTEI    [[YVL-TVTEI, YVLTVTEI, 0.0167, 5.2287, 0.2134,...
YVLTVTEI    [[YVL-TVTEI, YVLTVTEI, 0.0167, 5.2287, 0.2134,...
YVLTVTEI    [[YVL-TVTEI, YVLTVTEI, 0.0167, 5.2287, 0.2134,...
Length: 89, dtype: object

In [31]:
mhc.loc['YVLTVTEI']

base          HLA-A02:01                                              \
          Pos       ID       core     icore EL-score EL_Rank BA-score BA_Rank   
Peptide                                                                         
YVLTVTEI    0  PEPLIST  YVL-TVTEI  YVLTVTEI   0.0167  5.2287   0.2134  9.4989   
YVLTVTEI    0  PEPLIST  YVL-TVTEI  YVLTVTEI   0.0167  5.2287   0.2134  9.4989   

         HLA-B07:02                                                  end     
               core     icore EL-score  EL_Rank BA-score BA_Rank     Ave NB  
Peptide                                                                      
YVLTVTEI  YVLTV-TEI  YVLTVTEI   0.0011  14.4217    0.069  19.264  0.0089  0  
YVLTVTEI  YVLTV-TEI  YVLTVTEI   0.0011  14.4217    0.069  19.264  0.0089  0

In [66]:
test= remove_dupe_kmers(pd.read_csv('../out_test/merged.txt'))

In [68]:
len(test.peptide.unique())

10396771

In [86]:
tmp = cat.copy()
tmp['id_position'] = tmp['uniprot_id'] + '_' + tmp['start_pos'].astype(str)
# Find all duplicates and first duplicates index
total_index = tmp.loc[tmp['peptide'].duplicated(keep=False)].index
first_index = total_index.difference(tmp.loc[tmp.duplicated('peptide', keep='first')].index)

In [100]:
test[['peptide']].head(100).to_csv('test.pep', header=None, index=False)
pd.read_csv('test.pep', header=None)

,0
0,MSSHEGGKK
1,SSHEGGKKK
2,SHEGGKKKA
3,HEGGKKKAL
4,EGGKKKALK
...,...
95,SRYPDQWIV
96,RYPDQWIVP
97,YPDQWIVPG
98,PDQWIVPGG


In [110]:
dir_ = '../out_test/'
test = remove_dupe_kmers()
test2 = remove_dupe_kmers(dir_+'merged.txt')

ValueError: Length mismatch: Expected 7834499 rows, received array of length 116266

In [115]:
pd_merge = pd.concat([pd.read_csv(os.path.join(dir_, x)) for x in os.listdir(dir_) if 'split' in x and '.txt' in x], axis=0)

In [116]:
len(pd_merge.peptide.unique())

10396771

In [118]:
remove_dupe_kmers(pd_merge)

KeyboardInterrupt: 

In [119]:
test2 = pd.read_csv(dir_+'merged.txt')

C:\Users\richi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
len(pd_merge.peptide.unique()), len(test2.peptide.unique())

In [123]:
len(pd_merge.peptide.unique()), len(test2.peptide.unique())

(10396771, 10396772)

In [125]:
os.listdir('../data/human_split/')

['Human_split_0.fasta',
 'Human_split_1000.fasta',
 'Human_split_10000.fasta',
 'Human_split_11000.fasta',
 'Human_split_12000.fasta',
 'Human_split_13000.fasta',
 'Human_split_14000.fasta',
 'Human_split_15000.fasta',
 'Human_split_16000.fasta',
 'Human_split_17000.fasta',
 'Human_split_18000.fasta',
 'Human_split_19000.fasta',
 'Human_split_2000.fasta',
 'Human_split_20000.fasta',
 'Human_split_3000.fasta',
 'Human_split_4000.fasta',
 'Human_split_5000.fasta',
 'Human_split_6000.fasta',
 'Human_split_7000.fasta',
 'Human_split_8000.fasta',
 'Human_split_9000.fasta']